In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from scipy.stats import norm
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr
import calendar

In [3]:
df = pd.read_csv('data/weatherstats_vancouver_hourly_excelclean.csv')
df.head()

,date_time_local,pressure_station,pressure_sea,wind_dir,wind_dir_10s,wind_speed,wind_gust,relative_humidity,dew_point,temperature,windchill,humidex,visibility,air_quality_health_index,cloud_cover converted,max_air_temp_pst1hr,min_air_temp_pst1hr
0,2023-07-22 14:00:00 PDT,101.76,101.81,S,19.0,7,NaN,51.0,14.1,25.1,NaN,29.0,32200.0,2.0,25.0,25.4,24.4
1,2023-07-22 13:00:00 PDT,101.77,101.82,SSE,15.0,15,NaN,52.0,14.2,24.8,NaN,28.0,32200.0,1.7,37.5,24.9,22.2
2,2023-07-22 12:00:00 PDT,101.79,101.84,ESE,11.0,15,NaN,57.0,13.3,22.2,NaN,25.0,32200.0,1.5,87.5,22.7,20.3
3,2023-07-22 11:00:00 PDT,101.79,101.84,ESE,11.0,18,28.0,61.0,13.5,21.3,NaN,NaN,32200.0,1.3,75.0,21.3,18.6
4,2023-07-22 10:00:00 PDT,101.78,101.83,ESE,12.0,23,NaN,70.0,13.1,18.7,NaN,NaN,32200.0,1.1,100.0,19.6,17.8
